In [ ]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

In [ ]:
# Load the dataset
data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

In [ ]:
# This is a hack to avoid having to re - calculate weights when a cluster is split.
unscaled_inputs_all = data[:, 1:-1]
targets_all = data[:, -1]

In [ ]:
# Remove zero - target nodes. This is done by checking the number of targets in targets_all to see if there are zero - target nodes
num_one_targets = int(np.sum(targets_all))
zero_target_counter = 0 # Count the number of zero targets
indices_to_remove = []

for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_target_counter += 1
    if zero_target_counter > num_one_targets:
      indices_to_remove.append(i)

In [ ]:
# Remove the excess zero targets
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [ ]:
# Scale unscaled inputs to equal priors. This is useful for preprocessing that has been applied to a batch
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [ ]:
# Randomly shuffle the nodes and compute the targets. This is useful for testing and to avoid shuffling
suffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(suffled_indices)
shuffled_inputs = scaled_inputs[suffled_indices]
targets_shuffled = targets_equal_priors[suffled_indices]

In [ ]:
# Split the dataset into training validation and test sets and split the dataset into training, validation and test sets
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = targets_shuffled[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = targets_shuffled[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = targets_shuffled[train_samples_count+validation_samples_count:]

print(train_inputs.shape, train_targets.shape, validation_inputs.shape, validation_targets.shape, test_inputs.shape, test_targets.shape)

(3579, 10) (3579,) (447, 10) (447,) (448, 10) (448,)


In [ ]:
# Save Audiobooks data to npz files for use in tests. This is a helper function to avoid having to re - write the code
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [ ]:
# This is a copy of Audiobooks_data_test. npz but with a different
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float32)
train_targets = npz['targets'].astype(np.int32)

npz= np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float32)
validation_targets = npz['targets'].astype(np.int32)

npz = np.load('Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float32)
test_targets = npz['targets'].astype(np.int32)

In [ ]:
# Creates a Sequential model that can be used to train and test consensus. The input_size is the number of features in the input layer
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [ ]:
# Compile and return Keras optimizer
model. compile to avoid having to recompile
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Trains and returns the model
batch_size = 100
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, train_targets, batch_size=batch_size, epochs=max_epochs, validation_data=(validation_inputs, validation_targets), callbacks=[early_stopping], verbose=2)

Epoch 1/100
36/36 - 1s - loss: 0.6863 - accuracy: 0.5407 - val_loss: 0.6708 - val_accuracy: 0.6734 - 1s/epoch - 38ms/step
Epoch 2/100
36/36 - 0s - loss: 0.6623 - accuracy: 0.6868 - val_loss: 0.6489 - val_accuracy: 0.7204 - 103ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.6342 - accuracy: 0.7184 - val_loss: 0.6202 - val_accuracy: 0.7472 - 126ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.5991 - accuracy: 0.7527 - val_loss: 0.5852 - val_accuracy: 0.7315 - 120ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.5596 - accuracy: 0.7488 - val_loss: 0.5480 - val_accuracy: 0.7360 - 123ms/epoch - 3ms/step
Epoch 6/100
36/36 - 0s - loss: 0.5191 - accuracy: 0.7625 - val_loss: 0.5125 - val_accuracy: 0.7517 - 105ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.4830 - accuracy: 0.7765 - val_loss: 0.4794 - val_accuracy: 0.7606 - 117ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.4530 - accuracy: 0.7793 - val_loss: 0.4549 - val_accuracy: 0.7651 - 104ms/epoch - 3ms/step
Epoch 9/10

In [ ]:
# Tests the model on test_inputs and test_targets. Returns test_loss and test_accuracy
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3416 - accuracy: 0.8125


In [ ]:
# Prints test loss and accuracy to standard output
print('Test loss: {0:.2f}. Test Accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100.))

Test loss: 0.34. Test Accuracy: 81.25%
